## Objectives

This script allows to download:
1) spice kernels
2) odf and ifms files (closed/open loop data)
3) ancillary files such as tropospheric and ionospheric data 

List of Supported input_missions (this list will grow):

- JUICE
- MRO
- LRO
- GRAIL
- Insight
- Cassini

In [1]:
# Load required standard modules
import multiprocessing as mp
import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import axes3d

# Load required tudatpy modules
from tudatpy import constants
from tudatpy.io import save2txt
# from tudatpy.io import grail_mass_level_0_file_reader
# from tudatpy.io import grail_antenna_file_reader
from tudatpy.interface import spice
from tudatpy import numerical_simulation
from tudatpy.astro import time_conversion
from tudatpy.astro import frame_conversion
from tudatpy.astro import element_conversion
from tudatpy.math import interpolators
from tudatpy.numerical_simulation import environment_setup
from tudatpy.numerical_simulation import propagation
from tudatpy.numerical_simulation.environment_setup import radiation_pressure
from tudatpy.numerical_simulation import propagation_setup
from tudatpy.numerical_simulation import estimation, estimation_setup
from tudatpy.numerical_simulation import create_dynamics_simulator
from tudatpy.numerical_simulation.estimation_setup import observation
from tudatpy import util

/var/folders/vw/h3bwl54d4h9fxt45xnkxm86jgk0dgk/T/ipykernel_74094/1516825575.py:11: DeprecationWarning: Importing from the `tudatpy.io` module is deprecated since tudatpy 0.7 and will raise an error two minor versions hence. To eliminate this warning import from `tudatpy.data` instead.
  from tudatpy.io import save2txt


In [2]:
from more_generic_load_pds_files import download_url_files_time, dynamic_download_url_files_time_interval
from datetime import datetime, timedelta
from urllib.request import urlretrieve

from more_generic_load_pds_files_class import *

In [3]:
object = LoadPDS()

In [4]:
current_directory = os.getcwd()

In [5]:
# SUPPORTED PATTERNS: note MEX is missing for now. 
print(object.Orientation.supported_orientation_formats.keys())
print(object.Orientation.supported_orientation_formats.values())

dict_keys(['juice', 'mro', 'lro', 'cassini'])
dict_values(['^(?P<mission>juice)_(?P<instrument>(sc|mga|sa|lpbooms|magboom|majis_scan|swi_scan))_(?P<data_type>(attc|attm|meas|cmmd|plan|ptr|crema_[A-Z]+))_(?P<start_date_file>\\d{6})_(?P<end_date_file>\\d{6})_(?P<sclk>(s|t|f)\\d{6})_(?P<version>v\\d{2})(?P<extension>\\.bc)$', '^(?P<mission>mro)_(?P<instrument>[a-z]+)(?:_(?P<purpose>[a-z]+))?_(?P<start_date_file>\\d{6})_(?P<end_date_file>\\d{6})(?P<extension>\\.bc)$', '^(lro)([a-z]+)?(?:_([a-z]+))?_(\\d{6}|\\d{7})_(\\d{6}|\\d{7})(?:_(s\\d{6}))?(?:_(v\\d+))?(\\.bc)$', '^(?:([a-zA-Z0-9]+)_)?(\\d{5})_(\\d{5})([a-z]{0,2})?(\\.bc)$'])


In [6]:
#EXAMPLE OF ADDING A USER-CUSTOM PATTERN 
# WE ADD THE PATTERN CORRESPONDING TO MEX (previously missing). ONCE IT IS ADDED TO THE SUPPORTED PATTERNS, IT CAN BE USED
object.add_custom_mission_pattern("mex",
                    "^(?P<data>ATNM)_(?P<purpose>MEASURED)_(?P<start_date_file>\d{6})_(?P<end_date_file>\d{6})_(?P<version>(V\d+))?(?P<extension>\.BC)$")

print(object.Orientation.supported_orientation_formats.keys())
print(object.Orientation.supported_orientation_formats.values())

dict_keys(['juice', 'mro', 'lro', 'cassini', 'mex'])
dict_values(['^(?P<mission>juice)_(?P<instrument>(sc|mga|sa|lpbooms|magboom|majis_scan|swi_scan))_(?P<data_type>(attc|attm|meas|cmmd|plan|ptr|crema_[A-Z]+))_(?P<start_date_file>\\d{6})_(?P<end_date_file>\\d{6})_(?P<sclk>(s|t|f)\\d{6})_(?P<version>v\\d{2})(?P<extension>\\.bc)$', '^(?P<mission>mro)_(?P<instrument>[a-z]+)(?:_(?P<purpose>[a-z]+))?_(?P<start_date_file>\\d{6})_(?P<end_date_file>\\d{6})(?P<extension>\\.bc)$', '^(lro)([a-z]+)?(?:_([a-z]+))?_(\\d{6}|\\d{7})_(\\d{6}|\\d{7})(?:_(s\\d{6}))?(?:_(v\\d+))?(\\.bc)$', '^(?:([a-zA-Z0-9]+)_)?(\\d{5})_(\\d{5})([a-z]{0,2})?(\\.bc)$', '^(?P<data>ATNM)_(?P<purpose>MEASURED)_(?P<start_date_file>\\d{6})_(?P<end_date_file>\\d{6})_(?P<version>(V\\d+))?(?P<extension>\\.BC)$'])


In [7]:
# Test examples Orientation Files
MRO_dict, MRO_underscores = object.Orientation.parse_orientation_filename("mro", "mro_hga_psp_060311_060331.bc")
JUICE_dict, JUICE_underscores = object.Orientation.parse_orientation_filename("juice", "juice_mga_meas_230701_230719_s230628_v01.bc")
MEX_dict, MEX_underscores = object.Orientation.parse_orientation_filename("mex","ATNM_MEASURED_100101_110101_V03.BC")
#LRO_dict = object.Orientation.parse_orientation_filename("lro", "lrosc_2009186_2009213_v01.bc")
#CASSINI_dict = object.Orientation.parse_orientation_filename("cassini", "00017_00024rb.bc")

print(object.Orientation.reconstruct_orientation_filename(JUICE_dict, JUICE_underscores))
print(object.Orientation.reconstruct_orientation_filename(MRO_dict,MRO_underscores))
print(object.Orientation.reconstruct_orientation_filename(MEX_dict,MEX_underscores))
#print(object.Orientation.reconstruct_orientation_filename("lro",LRO_dict))
#print(object.Orientation.reconstruct_orientation_filename("cassini", CASSINI_dict)) 

juice_mga_meas_230701_230719_s230628_v01.bc
mro_hga_psp_060311_060331.bc
ATNM_MEASURED_100101_110101_V03.BC


In [8]:
# Test examples Orientation Files
#LRO_RS_dict = object.RadioScience.parse_RS_filename("lro", "lrosc_2009186_2009213_v01.bc")
#LRO_reconstructed_filename = object.RadioScience.reconstruct_RS_filename("lro", LRO_RS_dict))
#CASSINI_RS_dict = object.RadioScience.parse_RS_filename("cassini", "00017_00024rb.bc")
#CASSINI_reconstructed_filename = object.RadioScience.reconstruct_RS_filename("cassini", CASSINI_RS_dict))

MRO_RS_dict, MRO_RS_underscores = object.RadioScience.parse_RS_filename("mro","mromagr2006_242_0216xmmmv1.odf")
print(MRO_RS_dict)
MRO_reconstructed_filename = object.RadioScience.reconstruct_RS_filename(MRO_RS_dict, MRO_RS_underscores)
print(MRO_reconstructed_filename)

MEX_RS_dict, MEX_RS_underscores = object.RadioScience.parse_RS_filename("mex","m32icl1l02_d2x_031801831_00.tab")
MEX_reconstructed_filename = object.RadioScience.reconstruct_RS_filename(MEX_RS_dict, MEX_RS_underscores)
print(MEX_RS_dict)
print(MEX_reconstructed_filename)

## Commenting JUICE, since we do not have the fdets downloader/reader yet, and it will produce the following error:
## ''Selected Mission Not Supported (yet!) Aborting ...''
#JUICE_RS_dict = object.RadioScience.parse_RS_filename("juice", "juice_mga_meas_230701_230719_s230628_v01.bc")
#print(object.Orientation.reconstruct_RS_filename("juice",JUICE_RS_dict))

{'mission': 'mro', 'dataset': 'magr', 'date_file': '2006_242_0216', 'uplink': 'x', 'station': 'mm', 'downlink': 'm', 'version': 'v1', 'extension': '.odf'}
mromagr2006_242_0216xmmmv1.odf
{'mission': 'm32icl1l02', 'band': 'd2x', 'date_file': '031801831', 'version': '00', 'extension': '.tab'}
m32icl1l02_d2x_031801831_00.tab


In [9]:
def get_mex_files(local_path, start_date, end_date):
    input_mission = 'mex'
    all_dates = [start_date+timedelta(days=x) for x in range((end_date-start_date).days+1)]

    # Clock files
    print('---------------------------------------------')
    print('Download MEX clock files')
    clock_files=["mex_041215_step.tsc"] #latest mex tsc file
    url_clock_files="https://pds-geosciences.wustl.edu/mex/mex-m-mrs-1_2_3-v1/mexmrs_0001/extras/ancillary/spice/sclk/"
    for file in clock_files:
        if ( os.path.exists(local_path+file) == False ):
            print('download', local_path+file)
            urlretrieve(url_clock_files+file, local_path+file)

    print('relevant clock files')
    for k in range(len(clock_files)):
        clock_files[ k ] = local_path+clock_files[k]
        print(clock_files[k])

    # Orientation files
    print('---------------------------------------------')
    print('Download MEX orientation kernels')
    url_orientation_files = "https://spiftp.esac.esa.int/data/SPICE/MARS-EXPRESS/kernels/ck/"
    orientation_files_to_load = object.dynamic_download_url_files_time_interval(input_mission, 
        local_path=local_path, filename_format='ATNM_MEASURED_*.BC', start_date=start_date,end_date=end_date,
        url=url_orientation_files, time_interval_format='%y%m%d_%y%m%d')
    
    #print('---------------------------------------------')
    #print('Download MEX Solar Array kernels')
    #antenna_files_to_load = object.try_dynamic_download_url_files_time_interval(
    #    local_path=local_path, filename_format='juice_sa_meas_*.bc', start_date=start_date,end_date=end_date,
    #    url=url_orientation_files, time_interval_format='%y%m%d_%y%m%d')

    #for file in antenna_files_to_load:
    #    orientation_files_to_load.append(file)

    #print('relevant orientation files')
    #for f in orientation_files_to_load:
    #    print(f)


    # # Tropospheric corrections
    # print('---------------------------------------------')
    # print('Download JUICE tropospheric corrections files')
    # url_tro_files = "https://pds-geosciences.wustl.edu/mro/mro-m-rss-1-magr-v1/mrors_0xxx/ancillary/tro/"
    # tro_files_to_load = download_url_files_time_interval(
    #     local_path=local_path, filename_format='mromagr*.tro', start_date=start_date,
    #     end_date=end_date, url=url_tro_files, time_interval_format='%Y_%j_%Y_%j' )

    # print('relevant tropospheric corrections files')
    # for f in tro_files_to_load:
    #     print(f)


    # # Ionospheric corrections
    # print('---------------------------------------------')
    # print('Download JUICE ionospheric corrections files')
    # url_ion_files = "https://pds-geosciences.wustl.edu/mro/mro-m-rss-1-magr-v1/mrors_0xxx/ancillary/ion/"
    # ion_files_to_load = download_url_files_time_interval(local_path=local_path, filename_format='mromagr*.ion', start_date=start_date,
    #                                  end_date=end_date, url=url_ion_files, time_interval_format='%Y_%j_%Y_%j' )

    # print('relevant ionospheric corrections files')
    # for f in ion_files_to_load:
    #     print(f)


    # IFMS files
    print('---------------------------------------------')
    print('Download MEX IFMS files')
    url_rs = ("https://pds-geosciences.wustl.edu/mex/mex-m-mrs-1_2_3-v1/mexmrs_0001/data/level02/closed_loop/ifms/dp2/")
    rs_files_to_load = object.dynamic_download_url_files_single_time(input_mission, 
        local_path, 'm32icl*.tab', start_date, end_date,
        url_rs,'%y%j%H%M')

    print('Relevant Downloaded Radio Science Files:')
    for f in rs_files_to_load:
        print(f)

    return rs_files_to_load, clock_files, orientation_files_to_load #, tro_files_to_load, ion_files_to_load

In [10]:
def get_juice_files(local_path, start_date, end_date):
    input_mission = 'juice'
    all_dates = [start_date+timedelta(days=x) for x in range((end_date-start_date).days+1)]

    # Clock files
    print('---------------------------------------------')
    print('Download JUICE clock files')
    clock_files=["juice_step_20160326_v03.tsc"] #latest juice tsc file
    url_clock_files="https://spiftp.esac.esa.int/data/SPICE/JUICE/kernels/sclk/"
    for file in clock_files:
        if ( os.path.exists(local_path+file) == False ):
            print('download', local_path+file)
            urlretrieve(url_clock_files+file, local_path+file)

    print('relevant clock files')
    for k in range(len(clock_files)):
        clock_files[ k ] = local_path+clock_files[k]
        print(clock_files[k])

    # Orientation files
    print('---------------------------------------------')
    print('Download JUICE orientation kernels')
    url_orientation_files = "https://spiftp.esac.esa.int/data/SPICE/JUICE/kernels/ck/"
    orientation_files_to_load = object.dynamic_download_url_files_time_interval(input_mission,
        local_path=local_path, filename_format='juice_sc_meas_*.bc', start_date=start_date,end_date=end_date,
        url=url_orientation_files, time_interval_format='%y%m%d_%y%m%d')

    print('---------------------------------------------')
    print('Download JUICE Solar Array kernels')
    antenna_files_to_load = object.dynamic_download_url_files_time_interval(input_mission,
        local_path=local_path, filename_format='juice_sa_meas_*.bc', start_date=start_date,end_date=end_date,
        url=url_orientation_files, time_interval_format='%y%m%d_%y%m%d')

    for file in antenna_files_to_load:
        orientation_files_to_load.append(file)

    #print('relevant orientation files')
    #for f in orientation_files_to_load:
    #    print(f)


    # # Tropospheric corrections
    # print('---------------------------------------------')
    # print('Download JUICE tropospheric corrections files')
    # url_tro_files = "https://pds-geosciences.wustl.edu/mro/mro-m-rss-1-magr-v1/mrors_0xxx/ancillary/tro/"
    # tro_files_to_load = download_url_files_time_interval(
    #     local_path=local_path, filename_format='mromagr*.tro', start_date=start_date,
    #     end_date=end_date, url=url_tro_files, time_interval_format='%Y_%j_%Y_%j' )

    # print('relevant tropospheric corrections files')
    # for f in tro_files_to_load:
    #     print(f)


    # # Ionospheric corrections
    # print('---------------------------------------------')
    # print('Download JUICE ionospheric corrections files')
    # url_ion_files = "https://pds-geosciences.wustl.edu/mro/mro-m-rss-1-magr-v1/mrors_0xxx/ancillary/ion/"
    # ion_files_to_load = download_url_files_time_interval(local_path=local_path, filename_format='mromagr*.ion', start_date=start_date,
    #                                  end_date=end_date, url=url_ion_files, time_interval_format='%Y_%j_%Y_%j' )

    # print('relevant ionospheric corrections files')
    # for f in ion_files_to_load:
    #     print(f)


    # IFMS files
    # print('---------------------------------------------')
    # print('Download JUICE IFMS files')
    # url_odf = ("https://pds-geosciences.wustl.edu/mro/mro-m-rss-1-magr-v1/mrors_0xxx/odf/")
    # odf_files_to_load = download_url_files_time(
    #     local_path=local_path, filename_format='mromagr*_\w\w\w\wxmmmv1.odf', start_date=start_date,
    #     end_date=end_date, url=url_odf, time_format='%Y_%j', filename_size=30, indices_date_filename=[7])

    # print('relevant odf files')
    # for f in odf_files_to_load:
    #     print(f)

    return clock_files, orientation_files_to_load #, tro_files_to_load, ion_files_to_load, odf_files_to_load

In [11]:
def get_mro_files(local_path, start_date, end_date):
    input_mission = 'mro'
    all_dates = [start_date+timedelta(days=x) for x in range((end_date-start_date).days+1)]

    # Clock files
    print('---------------------------------------------')
    print('Download MRO clock files')
    clock_files=["mro_sclkscet_00112_65536.tsc"]
    url_clock_files="https://naif.jpl.nasa.gov/pub/naif/pds/data/mro-m-spice-6-v1.0/mrosp_1000/data/sclk/"
    for file in clock_files:
        if ( os.path.exists(local_path+file) == False ):
            print('download', local_path+file)
            urlretrieve(url_clock_files+file, local_path+file)

    print('relevant clock files')
    for k in range(len(clock_files)):
        clock_files[ k ] = local_path+clock_files[k]
        print(clock_files[k])

    # Orientation files
    print('---------------------------------------------')
    print('Download MRO orientation kernels')
    url_orientation_files = "https://naif.jpl.nasa.gov/pub/naif/pds/data/mro-m-spice-6-v1.0/mrosp_1000/data/ck/"
    orientation_files_to_load = object.dynamic_download_url_files_time_interval(input_mission,
        local_path=local_path, filename_format='mro_sc_psp_*.bc', start_date=start_date,end_date=end_date,
        url=url_orientation_files, time_interval_format='%y%m%d_%y%m%d')
    print('Download MRO High Gain Antenna kernels')
    antenna_files_to_load = object.dynamic_download_url_files_time_interval(input_mission,
        local_path=local_path, filename_format='mro_hga_psp_*.bc', start_date=start_date,end_date=end_date,
        url=url_orientation_files, time_interval_format='%y%m%d_%y%m%d')

    for file in antenna_files_to_load:
        orientation_files_to_load.append(file)

    print('relevant orientation files')
    for f in orientation_files_to_load:
        print(f)


    # Tropospheric corrections
    #print('---------------------------------------------')
    #print('Download MRO tropospheric corrections files')
    #url_tro_files = "https://pds-geosciences.wustl.edu/mro/mro-m-rss-1-magr-v1/mrors_0xxx/ancillary/tro/"
    #tro_files_to_load = object.dynamic_download_url_files_time_interval(input_mission,
    #    local_path=local_path, filename_format='mromagr*.tro', start_date=start_date,
    #    end_date=end_date, url=url_tro_files, time_interval_format='%Y_%j_%Y_%j')

    #print('relevant tropospheric corrections files')
    #for f in tro_files_to_load:
    #    print(f)
        
    # Ionospheric corrections
    #print('---------------------------------------------')
    #print('Download MRO ionospheric corrections files')
    #url_ion_files = "https://pds-geosciences.wustl.edu/mro/mro-m-rss-1-magr-v1/mrors_0xxx/ancillary/ion/"
    #ion_files_to_load = object.dynamic_download_url_files_time_interval(input_mission, local_path=local_path, filename_format='mromagr*.ion', start_date=start_date,
    #                                end_date=end_date, url=url_ion_files, time_interval_format='%Y_%j_%Y_%j' )

    #print('relevant ionospheric corrections files')
    #for f in ion_files_to_load:
    #    print(f)


    # ODF files
    print('---------------------------------------------')
    print('Download MRO ODF files')
    url_rs = ("https://pds-geosciences.wustl.edu/mro/mro-m-rss-1-magr-v1/mrors_0xxx/odf/")
    rs_files_to_load = object.dynamic_download_url_files_single_time(input_mission,
        local_path=local_path, filename_format='mromagr*.odf', start_date=start_date,
        end_date=end_date, url=url_rs, time_format='%Y_%j')

    print('Relevant Downloaded Radio Science Files:')
    for f in rs_files_to_load:
        print(f)

    return clock_files, orientation_files_to_load, rs_files_to_load

In [12]:
if __name__ == "__main__":
    
    inputs = []

    start_date_juice = datetime(2023, 7, 1)
    end_date_juice = datetime(2023, 7, 10)
    
    start_date_mro = datetime(2007, 1, 3)
    end_date_mro = datetime(2007, 1, 5)

    start_date_mex = datetime(2003, 1, 1)
    end_date_mex = datetime(2004, 1, 10) 
    
    clock_files_to_load_mro, orientation_files_to_load_mro, rs_files_to_load_mro = get_mro_files("mro_kernels/", start_date_mro, end_date_mro)         
    clock_files_to_load_mex, orientation_files_to_load_mex, rs_files_to_load_mex = get_mex_files("mex_kernels/", start_date_mex, end_date_mex)         
    clock_files_to_load_juice, orientation_files_to_load_juice = get_juice_files("juice_kernels/", start_date_juice, end_date_juice)         
    
    #clock_files_to_load_mro, orientation_files_to_load_mro, tro_files_to_load_mro, ion_files_to_load_mro, odf_files_to_load_mro = \
    #    get_mro_files("mro_kernels/", start_date_mro, end_date_mro)

---------------------------------------------
Download MRO clock files
relevant clock files
mro_kernels/mro_sclkscet_00112_65536.tsc
---------------------------------------------
Download MRO orientation kernels
The following files already exist in the folder: ['mro_kernels/mro_sc_psp_070102_070108.bc'] and will not be downloaded.
...Done.
Download MRO High Gain Antenna kernels
The following files already exist in the folder: ['mro_kernels/mro_hga_psp_070102_070108.bc'] and will not be downloaded.
...Done.
relevant orientation files
---------------------------------------------
Download MRO ODF files
The following files already exist in the folder: ['mro_kernels/mromagr2007_004_1801xmmmv1.odf', 'mro_kernels/mromagr2007_003_0845xmmmv1.odf'] and will not be downloaded.
Expected filename to download: mromagr2007_004_1801xmmmv1.odf
File Already Exists in the Folder ;)
Expected filename to download: mromagr2007_003_0845xmmmv1.odf
File Already Exists in the Folder ;)
Downloading: https://pds